In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, roc_auc_score, f1_score, recall_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE

import os

random_state = 3601

In [2]:
data_path_2009 = r'C:\Users\nogag\Documents\birocracy\PTSDClassifier\PTSD\Data\2009'
df_2009 = pd.read_excel(os.path.join(data_path_2009, "PTSD.xlsx"))


In [3]:
data_path_2016 = r'C:\Users\nogag\Documents\birocracy\PTSDClassifier\PTSD\Data\2016'
df_2016 = pd.read_csv(os.path.join(data_path_2016, "IDF_ABM_16.2.15_wide.csv"))


df_2016 = df_2016[~(df_2016['Wave']=='nov12')]


## features in the original data

In [4]:
trans_2016_2009_features = {
    'bagrut': 'highschool_diploma',
 'ADHD': 'ADHD',
 'Accuracy_threat_T1': 'T1Acc1t',
 'Accuracy_NT_T1': 'T1Acc1n',
 'Threat_Bias_T1': 'T1bias',
 'PHQ_T1': 'phq1',
 'Trait_T1': 'trait1',
 'State_T1': 'state1',
  'PCL_T1': 'PCL1',
 'PCL_T4': 'PCL3'}


In [5]:
df_2016['Wave'].unique()

array(['august12', 'august13'], dtype=object)

## append PCL intrusion features

In [ ]:
PCL_2009 = pd.read_csv(os.path.join(data_path_2009, "questionnaire_PCL1.csv"))
PCL_2016 = pd.read_csv(os.path.join(data_path_2016, "questionnaire5_PCL.csv"))                      

intrusion_features_2009 = ["q6.1_INTRU", "q6.2_DREAM", "q6.3_FLASH", "q6.4_UPSET", "q6.5_PHYS"]
intrusion_features_2016 = ["q5.1", "q5.2", "q5.3", "q5.4", "q5.5"]
df_2009 = df_2009.merge(PCL_2009[intrusion_features_2009 + ["ID"]], on="ID", how='outer')
df_2016 = df_2016.merge(PCL_2016[intrusion_features_2016 + ["ID"]], on="ID", how='outer')

for i, j in zip(intrusion_features_2009, intrusion_features_2016):
    trans_2016_2009_features[j] = i

## append PHQ9 features

In [ ]:
PHQ9_2009 = pd.read_csv(os.path.join(data_path_2009, "questionnaire5_PHQ9.csv"))
PHQ9_2016 = pd.read_csv(os.path.join(data_path_2016, "questionnaire4_PHQ9.csv"))                      

PHQ9_features_2009 = ["T1q5.1", "T1q5.2", "T1q5.3", "T1q5.4", "T1q5.5", "T1q5.6", "T1q5.7", "T1q5.8", "T1q5.9"]
PHQ9_features_2016 = ["q4.1", "q4.2","q4.3", "q4.4", "q4.5", "q4.6", "q4.7", "q4.8", "q49"]
df_2009 = df_2009.merge(PHQ9_2009[PHQ9_features_2009 + ["ID"]], on="ID", how='outer')
df_2016 = df_2016.merge(PHQ9_2016[PHQ9_features_2016 + ["ID"]], on="ID", how='outer')

for i, j in zip(PHQ9_features_2009, PHQ9_features_2016):
    trans_2016_2009_features[j] = i

## append trait featues

In [ ]:
Trait_2009 = pd.read_csv(os.path.join(data_path_2009, "questionnaire3_trait_anxiety_revised.csv"))
Trait_2016 = pd.read_csv(os.path.join(data_path_2016, "questionnaire3_trait_anxiety.csv"))                      

Trait_features_2009 = ["T1q3.1", "T1q3.2", "T1q3.3", "T1q3.4", "T1q3.5", "T1q3.6", "T1q3.7", "T1q3.8", "T1q3.9", "T1q3.10", "T1q3.11", "T1q3.12", "T1q3.13", "T1q3.14", "T1q3.15", "T1q3.16", "T1q3.17", "T1q3.18", "T1q3.19", "T1q3.20"]
Trait_features_2016 = ["q3.1", "q3.2", "q3.3", "q3.4", "q3.5", "q3.6", "q3.7", "q3.8", "q3.9", "q3.10", "q3.11", "q3.12", "q3.13", "q3.14", "q3.15", "q3.16", "q3.17", "q3.18", "q3.19", "q3.20"]
df_2009 = df_2009.merge(Trait_2009[Trait_features_2009 + ["ID"]], on="ID", how='outer')
df_2016 = df_2016.merge(Trait_2016[Trait_features_2016 + ["ID"]], on="ID", how='outer')

for i, j in zip(Trait_features_2009, Trait_features_2016):
    trans_2016_2009_features[j] = i

## append state featues

In [ ]:
State_2009 = pd.read_csv(os.path.join(data_path_2009, "questionnaire2_state_anxiety_revised.csv"))
State_2016 = pd.read_csv(os.path.join(data_path_2016, "questionnaire2_state_anxiety.csv"))                      

State_features_2009 = ["T1q2.1", "T1q2.2", "T1q2.3", "T1q2.4", "T1q2.5", "T1q2.6", "T1q2.7", "T1q2.8", "T1q2.9", "T1q2.10", "T1q2.11", "T1q2.12", "T1q2.13", "T1q2.14", "T1q2.15", "T1q2.16", "T1q2.17", "T1q2.18", "T1q2.19", "T1q2.20"]
State_features_2016 = ["q2.1", "q2.2", "q2.3", "q2.4", "q2.5", "q2.6", "q2.7", "q2.8", "q2.9", "q2.10", "q2.11", "q2.12", "q2.13", "q2.14", "q2.15", "q2.16", "q2.17", "q2.18", "q2.19", "q2.20"]
df_2009 = df_2009.merge(State_2009[State_features_2009 + ["ID"]], on="ID", how='outer')
df_2016 = df_2016.merge(State_2016[State_features_2016 + ["ID"]], on="ID", how='outer')

for i, j in zip(State_features_2009, State_features_2016):
    trans_2016_2009_features[j] = i

## target feature

In [6]:
target_feature = 'PCL3'
X_features = [i for i in trans_2016_2009_features.values() if not i == target_feature]


In [7]:
df_2016['PCL_T4'] = (df_2016['PCL_T4'] > 39).astype(int)
df_2009['PCL3'] = (df_2009['PCL3'] > 39).astype(int)

## adjust features from 2016

In [8]:
df_2016['bagrut'] = df_2016['bagrut'] == 'yes'
df_2016['dyslexia'] = df_2016['dyslexia'] == 'yes'
df_2016['ADHD'] = df_2016['ADHD'] == 'yes'

In [9]:
df_2016 = df_2016.rename(trans_2016_2009_features, axis=1)

In [10]:
df_2009 = df_2009[~df_2009[target_feature].isna()]
df_2016 = df_2016[~df_2016[target_feature].isna()]

## 2009 data outer CV

In [11]:
x, x_test, y, y_test = train_test_split(df_2009[X_features], df_2009[target_feature], test_size=0.25,
                                        random_state=random_state, stratify=df_2009[target_feature])

## parameters init

In [12]:
cv = StratifiedKFold(6, random_state=random_state, shuffle=True)

# class weight
pos_sample = y.sum() 
all_samples = y.count()
class_weights = all_samples/ pos_sample


# pipeline

pipe = Pipeline(steps=[
   # ('RFE', RFE(CatBoostClassifier(verbose=0, random_state=random_state))),
    ('classifier', CatBoostClassifier(verbose=0, random_state=random_state))])
grid_params = [{
 #       'RFE__n_features_to_select': [15],
        'classifier__class_weights':[[1, class_weights], [1, class_weights*2], [1, class_weights*0.5]],
        'classifier__l2_leaf_reg': np.arange(1,10),
        'classifier__depth': [4, 6, 8]#, 9]
        }]




## 2016

In [13]:
x_2016, y_2016 = df_2016[df_2016['Wave']=='august13'][X_features+['Intrusion_T1']], df_2016[df_2016['Wave']=='august13'][target_feature]
clf = GridSearchCV(pipe, grid_params, cv=cv, scoring='roc_auc')


In [14]:
x_2012, y_2012 = df_2016[df_2016['Wave']=='august12'][X_features+['Intrusion_T1']], df_2016[df_2016['Wave']=='august12'][target_feature]

clf.fit(x_2012, y_2012.astype(int), classifier__early_stopping_rounds = 15)
print(f"roc_auc = {clf.best_score_}, params = {clf.best_params_}")

roc_auc = 0.6909019343229869, params = {'classifier__class_weights': [1, 6.361538461538461], 'classifier__depth': 6, 'classifier__l2_leaf_reg': 9}


In [15]:
y_pred_target = clf.best_estimator_.predict_proba(x_2016)[:, 1]
print(f"roc_auc = {roc_auc_score(y_2016.astype(int), y_pred_target)}")

roc_auc = 0.6316466552315609


In [16]:
for i, j in zip(X_features, clf.best_estimator_['classifier'].get_feature_importance()):
    print(i, j)

highschool_diploma 8.194818187848735
ADHD 5.829346885576336
T1Acc1t 10.259808263136863
T1Acc1n 8.375020832495368
T1bias 16.141649004582693
phq1 8.241827811346214
trait1 18.71452755375539
state1 6.8744564014423695
PCL1 11.281792458500677
